In [1]:
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import numpy as np


In [2]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\MUNIC")

In [3]:
df18=pd.read_csv("MUNIC_educacao_2018.csv", sep=';')
df21=pd.read_csv("MUNIC_educacao_2021.csv", sep=';')

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_15636\3916205321.py:1: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df18=pd.read_csv("MUNIC_educacao_2018.csv", sep=';')


In [4]:
df21['raca_cor']

0        Parda
1        Parda
2        Parda
3        Parda
4        Parda
         ...  
5565     Parda
5566    Branca
5567     Parda
5568     Parda
5569    Branca
Name: raca_cor, Length: 5570, dtype: object

In [5]:
df18 = df18[['Cod Municipio', 'genero', 'idade', 'raca_cor',
        'escolaridade']]
df21 = df21[['CodMun', 'genero', 'idade', 'raca_cor',
        'escolaridade']]
        

In [6]:
df18['ano']=2018
df21['ano']=2021

In [7]:
df18 = df18.rename(columns={'Cod Municipio':'id_municipio','CodMun':'id_municipio', 'escolaridade':'grau_instrucao'})
df21 = df21.rename(columns={'Cod Municipio':'id_municipio','CodMun':'id_municipio','escolaridade':'grau_instrucao'})

In [8]:
df18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id_municipio    5570 non-null   int64 
 1   genero          5570 non-null   object
 2   idade           5570 non-null   object
 3   raca_cor        5570 non-null   object
 4   grau_instrucao  5570 non-null   object
 5   ano             5570 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 261.2+ KB


In [9]:
df21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id_municipio    5570 non-null   int64 
 1   genero          5570 non-null   object
 2   idade           5570 non-null   object
 3   raca_cor        5570 non-null   object
 4   grau_instrucao  5570 non-null   object
 5   ano             5570 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 261.2+ KB


In [10]:
df = pd.concat([df18,df21])

In [11]:
df['idade']=np.where(df['idade']=='(*) Não soube informar',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Recusa',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Não informou',np.nan,df['idade'])
df['idade'] =pd.to_numeric(df['idade'])


In [12]:
limites = [0, 30, 50,65,100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias)


In [13]:
x = df.pivot_table(columns=['ano','genero', 'faixa_etaria', 'raca_cor',
        'grau_instrucao'], aggfunc='size')
y = pd.DataFrame(x)
y = y.reset_index() ##inserir a contagem como coluna
y = y.rename(columns={0:'count'})

In [14]:
df = y[y['count']>0]

In [15]:
sum(df[df['ano']==2021]['count'])

5558

In [16]:
# criando dicionário
dict_esco = {'Ensino fundamental (1º Grau) incompleto':'Até Ensino Fundamental',
             'Ensino fundamental (1º Grau) completo':'Até Ensino Fundamental',
             'Ensino médio (2º Grau) incompleto':'Até Ensino Médio',
             'Ensino médio (2º Grau) completo':'Até Ensino Médio',
             'Ensino superior incompleto':'Até Ensino Superior Completo',
             'Ensino superior completo':'Até Ensino Superior Completo',
             'Especialização':'Até Pós Graduação ou Mestrado',
             'Mestrado':'Até Pós Graduação ou Mestrado',
             'Doutorado':'Até Doutorado'}


In [17]:
df = df.replace({'grau_instrucao':dict_esco}) #substituindo valores para padronizar


In [18]:
df

,ano,genero,faixa_etaria,raca_cor,grau_instrucao,count
14,2018,Feminino,Entre 18-29,Branca,Até Ensino Médio,1
16,2018,Feminino,Entre 18-29,Branca,Até Ensino Superior Completo,26
17,2018,Feminino,Entre 18-29,Branca,Até Ensino Superior Completo,6
18,2018,Feminino,Entre 18-29,Branca,Até Pós Graduação ou Mestrado,25
19,2018,Feminino,Entre 18-29,Branca,Até Pós Graduação ou Mestrado,7
...,...,...,...,...,...,...
1820,2021,Masculino,Acima de 65,Parda,Até Ensino Superior Completo,6
1821,2021,Masculino,Acima de 65,Parda,Até Ensino Superior Completo,1
1822,2021,Masculino,Acima de 65,Parda,Até Pós Graduação ou Mestrado,2
1823,2021,Masculino,Acima de 65,Parda,Até Pós Graduação ou Mestrado,1


In [19]:
df=df.rename(columns={'count':'quantidade','raca_cor':'cor_raca'})

In [ ]:
### Subindo para o GBQ

In [ ]:
client = bigquery.Client()

In [ ]:
dataset_ref = client.dataset('cargos_lideranca')

In [ ]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
 bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
 bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação'),
 bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
 bigquery.SchemaField('grau_instrucao','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação'),
 bigquery.SchemaField('quantidade','INTEGER',description='Área deformação.')
 ]

In [ ]:
table_ref = dataset_ref.table('MUNIC_perfil_gestor_educacao_consolidado')

In [ ]:
job_config = bigquery.LoadJobConfig(schema=schema)

In [ ]:
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=30db6602-a1b4-4c66-a1d1-5a76c91f2372>